In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import yaml
import cv2
from skimage.io import imread
from skimage.io import imsave
import matplotlib
import matplotlib.pyplot as plt

In [6]:
thresholds = {'cy3': 300,'cy5':500}
channels = ['cy3','cy5']
intensity_df = pd.read_csv('./intensity_test.csv')

In [9]:
def get_seq_df(intensity_df,thresholds):
    coordinates = intensity_df[['Y','X']].to_numpy()
    bool_df = pd.DataFrame({'Y':coordinates[:,0],'X':coordinates[:,1]})
    for cyc in range(1,10+1):
        for channel in channels:
            bool_df[f'cyc_{cyc}_{channel}'] = intensity_df[f'cyc_{cyc}_{channel}']>=thresholds[channel]# *decay[i]
    base_bool_map = {(True,True):'A',(True,False):'T',(False,True):'C',(False,False):'G'}
    base_df = pd.DataFrame({'Y':coordinates[:,0],'X':coordinates[:,1]})
    for cyc in tqdm(range(1,1+10),desc='Base calling'):
        base_df[f'cyc_{cyc}'] = bool_df.apply(lambda x: get_base(x[f'cyc_{cyc}_cy3'],x[f'cyc_{cyc}_cy5']),axis=1)
    seq_df = pd.DataFrame({'Y':coordinates[:,0],'X':coordinates[:,1]})
    seq_df['Sequence'] = base_df[[f'cyc_{i}' for i in range(1,1+10)]].agg(''.join,axis=1)
    return seq_df

def get_base(x,y):
    if x == y == True:
        return 'A'
    elif x == True and y == False:
        return 'T'
    elif x == False and y == True:
        return 'C'
    else:
        return 'G'

In [12]:
seq_df = get_seq_df(intensity_df,thresholds)
seq_df.to_csv('intensity_test_out.csv')

Base calling: 100%|██████████| 10/10 [00:00<00:00, 909.49it/s]
